<a href="https://colab.research.google.com/github/rrrudolph/trade/blob/master/Strategy_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Format Incoming Data


In [70]:

import pandas as pd
import datetime as dt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/My Drive/Colab Notebooks/EURUSD15.csv')
df = df[1800:2800]

#### This stuff was to format an original MT4 OHLC file ####

# df = df.set_axis(['Date', 'Time','O','H','L','C','V'], axis=1, inplace=False)
# df['DT'] = df['Date'] + ' '+df['Time']
df['DT'] = pd.to_datetime(df['DT'])
# df.drop(columns=['Date','Time'],inplace=True)
df = df.reindex(columns=['DT', 'O', 'H', 'L', 'C', 'V', 'Bar', 'D_Range',
                         'ADR', 'Pct_Change', 'Frac_H', 'Frac_L', 'Locked_H', 'Locked_L',
                         'Sw_Price','Sw_Rating', 'Sw_Size', 'Sw_Pct', 'Sw_Weight',
                         'TZ_Active', 'TZ_Type', 'TZ_Start',
                         'TZ_End', 'TZ_H', 'TZ_L'])

# df['DT'] -= pd.Timedelta(hours=7) # minus 7 hours to match CST

# set bar type
df['Bar'][df['C'] < df['O']] = 'down'
df['Bar'][df['C'] > df['O']] = 'up'


# if you want to re-iterate over the swings uncomment these:
# df['Locked_H'] = 0
# df['Locked_L'] = 0

df.reset_index(drop=True,inplace=True)
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,Sw_Weight,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L
0,2017-08-23 20:15:00,1.18108,1.18139,1.18078,1.18139,634,up,0.00830,0.00900,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-23 20:30:00,1.18139,1.18159,1.18068,1.18076,670,down,0.00830,0.00900,NaN,4.0,1.0,1,1,NaN,3.0,-0.00092,-0.10,0.999875,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-23 20:45:00,1.18076,1.18121,1.18073,1.18120,445,up,0.00830,0.00900,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-23 21:00:00,1.18117,1.18119,1.18067,1.18097,494,down,0.00830,0.00900,NaN,0.0,3.0,1,1,NaN,2.0,0.00163,0.18,0.999755,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-23 21:15:00,1.18098,1.18126,1.18079,1.18097,539,down,0.00830,0.00901,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2017-09-07 05:00:00,1.19237,1.19251,1.19222,1.19249,700,up,0.01445,0.00907,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
996,2017-09-07 05:15:00,1.19250,1.19257,1.19231,1.19256,497,up,0.01445,0.00908,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
997,2017-09-07 05:30:00,1.19255,1.19280,1.19248,1.19255,622,NaN,0.01445,0.00909,NaN,6.0,0.0,1,1,NaN,3.0,-0.00141,-0.16,0.999786,NaN,NaN,NaN,NaN,NaN,NaN
998,2017-09-07 05:45:00,1.19255,1.19262,1.19243,1.19252,619,down,0.01445,0.00911,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DT          1000 non-null   datetime64[ns]
 1   O           1000 non-null   float64       
 2   H           1000 non-null   float64       
 3   L           1000 non-null   float64       
 4   C           1000 non-null   float64       
 5   V           1000 non-null   int64         
 6   Bar         985 non-null    object        
 7   D_Range     1000 non-null   float64       
 8   ADR         1000 non-null   float64       
 9   Pct_Change  0 non-null      float64       
 10  Frac_H      1000 non-null   float64       
 11  Frac_L      1000 non-null   float64       
 12  Locked_H    1000 non-null   int64         
 13  Locked_L    1000 non-null   int64         
 14  Sw_Price    0 non-null      float64       
 15  Sw_Rating   271 non-null    float64       
 16  Sw_Size     1000 non-null

# Find fractals, swings, and ADR.

In [0]:
# do I make one function do everything from formatting
# data to returning a trade signal? 

# def update_df(df):  

#### Find new fractals and get swing ratings ####

highs = df['H'].values
lows = df['L'].values

# Omit rows where the swings are locked and iterate over the 
# df using only the unlocked swings as starting points in the loop 
temp = df[(df['Locked_H'] != 1) & (df['Locked_L'] != 1)]
for i in temp.index:  
    low = lows[i]
    high = highs[i]

    # the order below is highs forwards then highs backwards

    try:   # iter forwards 
        count_prior = 0   # initialize counters and such
        count_next = 0
        next_ = i + 1
        prior_ = i - 1

        while high > highs[next_] and count_next < 80: # iter limit
            next_ +=1
            count_next +=1
        df.loc[i,'Locked_H'] = 1 # if it hasn't error'd lock the row 
    except:
        pass

    try: # iter backwards
        while high >= highs[prior_] and count_prior < 80: 
            prior_ -=1
            count_prior +=1
    except:
        pass 

    df.loc[i,'Frac_H'] = min([count_prior, count_next])

    # find the swing size
    try: # the 'min is an empty argument' requires try block
        if df.loc[i,'Frac_H'] > 1:  # this will return the swing count if frac > 1
            df.loc[i,'Sw_Size'] = min(lows[i+1:i+df.loc[i,'Frac_H']]) - high
    except:
        df.loc[i,'Sw_Size'] = 0

    ## now the same thing but for the lows

    try: # iter forwards 
        count_prior = 0
        count_next = 0
        next_ = i + 1
        prior_ = i - 1

        while low < lows[next_] and count_next < 80: 
            next_ +=1
            count_next +=1
        df.loc[i,'Locked_L'] = 1 # if it hasn't error'd set the lock value 
    except:
        pass

    try: # iter backwards
        while low <= lows[prior_] and count_prior < 80: 
            prior_ -=1
            count_prior +=1  
    except:
        pass
    df.loc[i,'Frac_L'] = min([count_prior, count_next])

    # find swing size (price movement from hi to low or low to hi)
    # only overwrite if the new value is greater 
    try:  
        if max(highs[i:i+count_next]) - low > abs(df.loc[i,'Sw_Size']):
            if df.loc[i,'Frac_L'] > 1:
                df.loc[i,'Sw_Size'] = (max(highs[i+1:df.loc[i,'Frac_L']]) - low)
    except:
        pass



#### Calculate the Swing Ratings #### 

# get the swing size in relation to ADR
df['Sw_Pct'] = round(df['Sw_Size']/df['ADR'], 2)

# create a function of a curved line that will act as a weight
# to use on the swing size. this will taper off the extreme
# swing sizes to balance out the swing ratings overall
df['Sw_Weight'] = -1.02 ** abs(df['Sw_Pct']*100) # function for curved line
weight = df['Sw_Weight'][abs(df['Sw_Weight'].values) > 0] # list of weights 

# first fill NaNs with mean of group
# normalize so its a weight between 0 and 1
# now that the weights are normalized, multiply the swing by its weight
# add the fractal rating to its swing and divide by 2

weight.fillna(value=weight.mean(), inplace=True) 
df['Sw_Weight'] = (weight - min(weight)) / (max(weight) - min(weight))
final = df['Sw_Size'] * df['Sw_Weight']
temp = df[abs(df['Sw_Size']) > 0]
df['Sw_Rating'] = round((abs(final) * 2000 + temp['Frac_H'] + temp['Frac_L']) / 2)
# technically adding both Frac_H and _L is not perfect but it will work for 99% of cases

df.drop(columns=['Sw_Weight'], inplace=True)

# save the swing prices
for i in df.index:
    a = df.loc[i,'H'][ df.loc[i,'Sw_Size'] < 0]
    b = df.loc[i,'L'][ df.loc[i,'Sw_Size'] > 0]
    if a.size > 0:
        df.loc[i,'Sw_Price'] = a
    elif b.size > 0: 
        df.loc[i,'Sw_Price'] = b


#### get the D_Range and ADR  ####

# To calculate this per day it needs to lag by 24 hrs 
# Otherwise the current day will always look small

temp = df[df['DT'] < dt.datetime.today()]  # return all days prior to today (for live data) 
for i in temp['DT'].dt.date.unique():   # iterate through each unique day
    group = df.groupby(df['DT'].dt.date).get_group(i)   # group data by day
    dayrange = max(group['H']) - min(group['L'])   
    df.loc[group.index,'D_Range'] = dayrange

# ADR
#  There are 96 15m candles per day and I want a 5 day average.  
#  This will give me an index error so it needs a try block
##### NOTE: this needs to be modified so it can remain a 5 day
#####       average given any timeframe
for i in temp.index:
    try:
        df.loc[i,'ADR'] = round(df.loc[i-480:i,'D_Range'].mean(),5)
    except:
        num_rows = df.iloc[i].index - df.iloc[0].index
        df.loc[i,'ADR'] = round(df.loc[i - num_rows:i,'D_Range'].mean(),5)

# lastly, fill today's blank ADR values with yesterdays
df['ADR'].fillna(method='ffill', inplace=True)


####  Set Pct_Change  ####
df['Pct_Change'] = df['C'].pct_change()

In [73]:
final*2000

0      0.000000
1     -1.834660
2      0.000000
3      3.241499
4      0.000000
         ...   
995    0.000000
996    0.000000
997   -2.806069
998    0.000000
999    1.516542
Length: 1000, dtype: float64

# Trade Zone Viz

In [74]:
# originally this was going to hold the SR zone code
# but I actually just want to run SR zones for HTFs
# so it will need a separate loop

# cols for currently valid trade zones
# TZ stands for trade zone
df['TZ_Active'] = np.nan
df['TZ_Type'] = np.nan
df['TZ_Start'] = np.nan
df['TZ_End'] = np.nan

# save zones for vis
plotx = [] # sell zones
ploty = []
plotxx = [] # buy zones
plotyy = []


temp =  df[(df['TZ_Active'] != False) &
                (df['Sw_Rating'] > 10)]

for i in temp.index: #

    swing = abs(df.loc[i, 'Sw_Size'])
    # zone_height =  round(swing*5000 * df.loc[i,'ADR'] / 74 , 4)
    zone_height = df.loc[i, 'Sw_Rating'] * df.loc[i,'ADR'] / 75

    # set a limit on zone size 
    a = df.loc[2, 'DT'] - df.loc[1, 'DT']  
    b = pd.Timedelta(days = 1)
    cpd = b / a         # candles per day
    max_size = df.loc[i, 'ADR'] / cpd * 42  # wont work for D1 TF 

    if zone_height > max_size:
        zone_height = max_size
    # print("zone height: ", zone_height, "\n max: ", max_size)
    if df.loc[i, 'Sw_Size'] < 0:       # swing high
        df.loc[i,'TZ_Type'] = 'sell'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price'] + zone_height     # upper price 
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price']     # lower    
    else:               # swing low
        df.loc[i,'TZ_Type'] = 'buy'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price']     
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price'] - zone_height    
    
    end = round(df.loc[i, 'Sw_Rating'] * 3 + i)   # now get the length of the zone
    if end >  len(df) - 1:      # set max value as the length of df
        end =  len(df) - 1
    # elif end > round(max_size * 10000):
    #     end = round(max_size * 10000)
    df.loc[i,'TZ_End'] = end
   
    # set a custom start too with 20% lag time
    start = i # + round((end - i) * 0.2)
    if start >  len(df) - 1:
        start =  len(df) - 1
    df.loc[i,'TZ_Start'] = start

    # set buffers - used for switching zone types
    df['Buf_Hi'] = df['TZ_H'] + (df['TZ_H'] - df['TZ_L']) * 0.25
    df['Buf_Lo'] = df['TZ_L'] - (df['TZ_H'] - df['TZ_L']) * 0.25
   
    ###  Change sell zone type
    if df.loc[i,'TZ_Type'] == 'sell':
        ## this section for plotting ##
        plot_sellx.append(df.loc[start, 'DT'])
        plot_sellx.append(df.loc[end, 'DT'])
        plot_sellx.append(df.loc[end, 'DT'])
        plot_sellx.append(df.loc[start, 'DT'])
        plot_sellx.append(df.loc[start, 'DT'])
        plot_sellx.append(None)
        plot_selly.append(df.loc[i,'TZ_L'])
        plot_selly.append(df.loc[i,'TZ_L'])
        plot_selly.append(df.loc[i,'TZ_H'])
        plot_selly.append(df.loc[i,'TZ_H'])
        plot_selly.append(df.loc[i,'TZ_L'])
        plot_selly.append(None)

        # look for a new high above zone
        new_high = df.loc[i:][df['H'] > df.loc[i,'Buf_Hi']]
        if new_high.size > 0:   # if found
           
            df.loc[i,'TZ_Type'] = 'buy' # and slide the zone down
            start = new_high.idxmin     # shift zone forward
            end = start + end * 0.75     # shrink zone length by 25%
            df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 
            df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2
            
            ## this section for plotting ##
            plot_sell_flipx.append(df.loc[start, 'DT'])
            plot_sell_flipx.append(df.loc[end, 'DT'])
            plot_sell_flipx.append(df.loc[end, 'DT'])
            plot_sell_flipx.append(df.loc[start, 'DT'])
            plot_sell_flipx.append(df.loc[start, 'DT'])
            plot_sell_flipx.append(None)
            plot_sell_flipy.append(df.loc[i,'TZ_L'])
            plot_sell_flipy.append(df.loc[i,'TZ_L'])
            plot_sell_flipy.append(df.loc[i,'TZ_H'])
            plot_sell_flipy.append(df.loc[i,'TZ_H'])
            plot_sell_flipy.append(df.loc[i,'TZ_L'])
            plot_sell_flipy.append(None) 

            ###  then invalidate zone  ###
            # look forward for a new low below zone
            # idx = new_high.index[0]+1   
            # new_low = df.loc[idx:][df['L'] < df.loc[i,'Buf_Lo']]
            # if new_low.size > 0:  
            #     df.loc[i,'TZ_Active'] = False  
                
    ###  Change buy zone type:
    if df.loc[i,'TZ_Type'] == 'buy':
        ## this section for plotting ##
        plot_buyx.append(df.loc[start, 'DT'])
        plot_buyx.append(df.loc[end, 'DT'])
        plot_buyx.append(df.loc[end, 'DT'])
        plot_buyx.append(df.loc[start, 'DT'])
        plot_buyx.append(df.loc[start, 'DT'])
        plot_buyx.append(None)
        plot_buyy.append(df.loc[i,'TZ_L'])
        plot_buyy.append(df.loc[i,'TZ_L'])
        plot_buyy.append(df.loc[i,'TZ_H'])
        plot_buyy.append(df.loc[i,'TZ_H'])
        plot_buyy.append(df.loc[i,'TZ_L'])
        plot_buyy.append(None)

        # look forward for a new low below zone
        new_low = df.loc[i:][df['L'] < df.loc[i,'Buf_Lo']]
        if new_low.size > 0:  

            df.loc[i,'TZ_Type'] = 'sell'
            start = new_high.idxmin     # shift zone forward
            end = start + end * 0.75     # shrink zone length by 25%
            df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] + zone_height / 2 
            df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] + zone_height / 2  

            # this section for plotting ##
            plot_buy_flipx.append(df.loc[start, 'DT'])
            plot_buy_flipx.append(df.loc[end, 'DT'])
            plot_buy_flipx.append(df.loc[end, 'DT'])
            plot_buy_flipx.append(df.loc[start, 'DT'])
            plot_buy_flipx.append(df.loc[start, 'DT'])
            plot_buy_flipx.append(None)
            plot_buy_flipy.append(df.loc[i,'TZ_L'])
            plot_buy_flipy.append(df.loc[i,'TZ_L'])
            plot_buy_flipy.append(df.loc[i,'TZ_H'])
            plot_buy_flipy.append(df.loc[i,'TZ_H'])
            plot_buy_flipy.append(df.loc[i,'TZ_L'])
            plot_buy_flipy.append(None)       

            ###  then invalidate zone  ###
            # look forward for a new hi above zone
            # idx = new_low.index[0]+1   
            # new_hi = df.loc[idx:][df.loc[idx:, 'H'] > df.loc[i,'Buf_Hi']]
            # if new_hi.size > 0:  
            #     df.loc[i,'TZ_Active'] = False  
    # print(df['DT'][df.index == df.loc[i,'TZ_Start']])
# final invalidation of trade zones, this time by expiry
# if current bar in df is past TZ_End, zone is expired
# df['TZ_Active'][df.index.loc[-1] > df['TZ_End'] = False    # uncomment for live data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



TypeError: ignored

### TZ Plots

In [0]:
# plot zones

fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]
ratings = round(df['Sw_Rating'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Sw_Rating'][df['Sw_Size'] > 0],0)


fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
    # yaxis= 'y2'
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
    # yaxis= 'y2'
))

# current zones in purple
fig.add_trace(go.Scatter(x=plot_buyx, y=plot_buyy, fill="toself"))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_buy_flipy, fill="toself"))
fig.add_trace(go.Scatter(x=plot_sellx, y=plot_selly, fill="toself"))
fig.add_trace(go.Scatter(x=plot_sell_flipx, y=plot_sell_flipy, fill="toself"))






fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


In [0]:
new_high

#test
this will combine the TZ viz with actual entries as the zones unfold in real time and flip types. it mergers the TZ Viz code with the Trade Setup code. so after each zone is labeled it will check to see if any candles exist within the zone. if yes, it will check for a signal. if yes, it will save that signal to a list for plotting. the zones will only plot if a trade occurs

In [87]:


active_zones = df[(df['TZ_H'] != np.nan) & # there is a zone
                  (df['TZ_Active'] != False)] 


df['TZ_Active'] = np.nan
df['TZ_Type'] = np.nan
df['TZ_Start'] = np.nan
df['TZ_End'] = np.nan

# save zones for vis
plot_buyx = [] 
plot_buyy = []
plot_buy_flipx = [] 
plot_buy_flipy = []
plot_sellx = [] 
plot_selly = []
plot_sell_flipx = [] 
plot_sell_flipy = []
trade_entries = []
dates = []
names = []

temp =  df[(df['TZ_Active'] != False) &
                (df['Sw_Rating'] > 10)]

for i in temp.index: #

    swing = abs(df.loc[i, 'Sw_Size'])
    # zone_height =  round(swing*5000 * df.loc[i,'ADR'] / 74 , 4)
    zone_height = df.loc[i, 'Sw_Rating'] * df.loc[i,'ADR'] / 75

    # set a limit on zone size 
    a = df.loc[2, 'DT'] - df.loc[1, 'DT']  
    b = pd.Timedelta(days = 1)
    cpd = b / a         # candles per day
    max_size = df.loc[i, 'ADR'] / cpd * 42  # wont work for D1 TF 

    if zone_height > max_size:
        zone_height = max_size
    # print("zone height: ", zone_height, "\n max: ", max_size)
    if df.loc[i, 'Sw_Size'] < 0:       # swing high
        df.loc[i,'TZ_Type'] = 'sell'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price'] + zone_height     # upper price 
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price']     # lower    
    else:               # swing low
        df.loc[i,'TZ_Type'] = 'buy'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price']     
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price'] - zone_height    
    
    end = round(df.loc[i, 'Sw_Rating'] * 3.5 + i)   # now get the length of the zone
    if end >  len(df) - 1:      # set max value as the length of df
        end =  len(df) - 1
    # elif end > round(max_size * 10000):
    #     end = round(max_size * 10000)
    df.loc[i,'TZ_End'] = end
   
    # set a custom start too with 20% lag time
    start = i # + round((end - i) * 0.2) (makes viz difficult)
    if start >  len(df) - 1:
        start =  len(df) - 1
    df.loc[i,'TZ_Start'] = start

    # set buffers - used for switching zone types
    df['Buf_Hi'] = df['TZ_H'] + (df['TZ_H'] - df['TZ_L']) * 0.25
    df['Buf_Lo'] = df['TZ_L'] - (df['TZ_H'] - df['TZ_L']) * 0.25

    # create a df to store rows in which a 
    # candle prints within the current zone
    zone_df = df.loc[start:end]
    zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                        (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
    
    ###  Sell zone type
    if df.loc[i,'TZ_Type'] == 'sell':
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles
        if len(zone_df) > 0:
                # this is for sells only
                signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'up')]
                if len(signal_a) > 0:
                    # in case of multiple signals, return highest
                    sell_price = max(signal_a['O'])
                    trade_entries.append(sell_price)
                    idx = signal_a['O'].idxmax() # get index of entry price candle
                    date = zone_df.loc[idx, 'DT']

                    # plot 
                    dates.append(date)
                    names.append('sell')
                    plot_sellx.append(df.loc[start, 'DT'])
                    plot_sellx.append(df.loc[end, 'DT'])
                    plot_sellx.append(df.loc[end, 'DT'])
                    plot_sellx.append(df.loc[start, 'DT'])
                    plot_sellx.append(df.loc[start, 'DT'])
                    plot_sellx.append(None)
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(df.loc[i,'TZ_H'])
                    plot_selly.append(df.loc[i,'TZ_H'])
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(None)


        # look for a new high above zone
        new_high = df.loc[start:end, 'H'][df['H'] > df.loc[i, 'Buf_Hi']]
        if len(new_high) > 0:   # if found
           
            df.loc[i,'TZ_Type'] = 'buy' 
            start = min(new_high.index)    # shift zone forward
            end = round((end - start) * 0.5 + end)  # shift zone forward by 50%
            if end > len(df) - 1:
                end = len(df) - 1
            df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 
            df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

            zone_df = df.loc[start:end]
            zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                            (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
            if len(zone_df) > 0:
                # this is for buys only
                    
                # check for zone invalidation
                # but plot any trades that happen
                # while zone is still valid
                new_low = df.loc[start:end, 'L'][df['L'] < df.loc[i, 'TZ_L']]
                if len(new_low) > 0: 
                    df.loc[i,'TZ_Active'] = False    
                    idx = new_low.idxmin() # index of first candle to breach  (THIS IS NOT RETURNING THE FIRST CANDLE THIS IS RETURNING THE LOWEST CANDLE!)
                    zone_df = zone_df.loc[start:idx] # shorten the zone_df now to stop at that candle
                signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'down')]
                if len(signal_a) > 0:
                    buy_price = min(signal_a['O'])
                    trade_entries.append(buy_price)
                    idx = signal_a['O'].idxmax()  # get highest price ( I DONT THINK THIS IS ACCURATE!)
                    date = df.loc[idx, 'DT']
                    
                    # ( I want the index of the lowest 'O' - not just the latest 'O')

                    # plot 
                    dates.append(date)
                    names.append('buy')            
                    plot_sell_flipx.append(df.loc[start, 'DT'])
                    plot_sell_flipx.append(df.loc[end, 'DT'])
                    plot_sell_flipx.append(df.loc[end, 'DT'])
                    plot_sell_flipx.append(df.loc[start, 'DT'])
                    plot_sell_flipx.append(df.loc[start, 'DT'])
                    plot_sell_flipx.append(None)
                    plot_sell_flipy.append(df.loc[i,'TZ_L'])
                    plot_sell_flipy.append(df.loc[i,'TZ_L'])
                    plot_sell_flipy.append(df.loc[i,'TZ_H'])
                    plot_sell_flipy.append(df.loc[i,'TZ_H'])
                    plot_sell_flipy.append(df.loc[i,'TZ_L'])
                    plot_sell_flipy.append(None)

            ###  then invalidate zone  ###
            # look forward for a new low below zone
            new_low = df.loc[start:end, 'L'][df['L'] < df.loc[i, 'TZ_L']]
            if len(new_low) > 0:   # if found
                df.loc[i,'TZ_Active'] = False  
                
    # ###  Change buy zone type:
    # if df.loc[i,'TZ_Type'] == 'buy':
    #     ## this section for plotting ##
    #     plot_buyx.append(df.loc[start, 'DT'])
    #     plot_buyx.append(df.loc[end, 'DT'])
    #     plot_buyx.append(df.loc[end, 'DT'])
    #     plot_buyx.append(df.loc[start, 'DT'])
    #     plot_buyx.append(df.loc[start, 'DT'])
    #     plot_buyx.append(None)
    #     plot_buyy.append(df.loc[i,'TZ_L'])
    #     plot_buyy.append(df.loc[i,'TZ_L'])
    #     plot_buyy.append(df.loc[i,'TZ_H'])
    #     plot_buyy.append(df.loc[i,'TZ_H'])
    #     plot_buyy.append(df.loc[i,'TZ_L'])
    #     plot_buyy.append(None)

    #     zone_df = df.loc[start:end]
    #     zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
    #                     (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
    #     if len(zone_df) > 0:
    #         # this is for buys only
    #         signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'down')]
    #         if len(signal_a) > 0:
    #             buy_price = min(signal_a['O'])
    #             trade_entries.append(buy_price)
    #             idx = signal_a['O'].idxmin()
    #             date = df.loc[idx, 'DT']
    #             dates.append(date)
    #             names.append('buy')

    #     # look forward for a new low below zone
    #     new_low = df.loc[start:end, 'L'][df['L'] < df.loc[i,'Buf_Lo']]
    #     if new_low.size > 0:  

    #         df.loc[i,'TZ_Type'] = 'sell'
    #         start = new_low.idxmin()     # shift zone forward
    #         end = round(start + end * 0.75)     # shrink zone length by 25%
    #         if end > len(df) - 1:
    #             end = len(df) - 1
    #         df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] + zone_height / 2 
    #         df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] + zone_height / 2  

    #         # this section for plotting ##
    #         plot_buy_flipx.append(df.loc[start, 'DT'])
    #         plot_buy_flipx.append(df.loc[end, 'DT'])
    #         plot_buy_flipx.append(df.loc[end, 'DT'])
    #         plot_buy_flipx.append(df.loc[start, 'DT'])
    #         plot_buy_flipx.append(df.loc[start, 'DT'])
    #         plot_buy_flipx.append(None)
    #         plot_buy_flipy.append(df.loc[i,'TZ_L'])
    #         plot_buy_flipy.append(df.loc[i,'TZ_L'])
    #         plot_buy_flipy.append(df.loc[i,'TZ_H'])
    #         plot_buy_flipy.append(df.loc[i,'TZ_H'])
    #         plot_buy_flipy.append(df.loc[i,'TZ_L'])
    #         plot_buy_flipy.append(None) 

    #         # create a df to store rows in which a 
    #         # candle prints within the current zone
    #         zone_df = df.loc[start:end]
    #         zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
    #                         (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
        
    #         # if candles are found, look for a candle with lower vol than 
    #         # original swing, or a series of 3 sequentially lower vol candles
    #         if len(zone_df) > 0:
    #                 # this is for sells only
    #                 signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'up')]   
    #                 if len(signal_a) > 0:
    #                     sell_price = max(signal_a['O'])
    #                     trade_entries.append(sell_price)
    #                     idx = signal_a['O'].idxmax()
    #                     date = df.loc[idx, 'DT']
    #                     dates.append(date)
    #                     names.append('sell')      

    #         ###  then invalidate zone  ###
    #         # look forward for a new hi above zone
    #         idx = new_low.index[0]+1   
    #         new_hi = df.loc[idx:][df.loc[idx:, 'H'] > df.loc[i,'Buf_Hi']]
    #         if new_hi.size > 0:  
    #             df.loc[i,'TZ_Active'] = False  

#### final invalidation of trade zones, this time by expiry
# if current bar in df is past TZ_End, zone is expired
# df['TZ_Active'][df.index.loc[-1] > df['TZ_End'] = False    # uncomment for live data

# plot_buyx = [] 
# plot_buyy = []
# plot_buy_flipx = [] 
# plot_buy_flipy = []
# plot_sellx = [] 
# plot_selly = []
# plot_sell_flipx = [] 
# plot_sell_flipy = []
# trade_entries = []
# dates = []
# names = []

fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])

# current zones in purple
fig.add_trace(go.Scatter(x=plot_buyx, y=plot_buyy, fill="toself"))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_buy_flipy, fill="toself"))
fig.add_trace(go.Scatter(x=plot_sellx, y=plot_selly, fill="toself"))
fig.add_trace(go.Scatter(x=plot_sell_flipx, y=plot_sell_flipy, fill="toself"))


fig.add_trace(go.Scatter(
    x=dates,
    y=trade_entries,
    mode="markers+text",
    name="Swing Ratings",
    text=names,
    textposition="top center",
    # fillcolor='blue'
    marker_size=10,
    marker_color='rgba(0, 0, 0, 1)'
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)

# f

In [124]:


active_zones = df[(df['TZ_H'] != np.nan) & # there is a zone
                  (df['TZ_Active'] != False)] 


df['TZ_Active'] = np.nan
df['TZ_Type'] = np.nan
df['TZ_Start'] = np.nan
df['TZ_End'] = np.nan

# save zones for vis
plot_buyx = [] 
plot_buyy = []
plot_buy_flipx = [] 
plot_buy_flipy = []
plot_sellx = [] 
plot_selly = []
plot_buy_flipx = [] 
plot_sell_flipy = []
trade_entries = []
dates = []
names = []

temp =  df[(df['TZ_Active'] != False) &
                (df['Sw_Rating'] > 10)]

for i in temp.index: #

    swing = abs(df.loc[i, 'Sw_Size'])
    # zone_height =  round(swing*5000 * df.loc[i,'ADR'] / 74 , 4)
    zone_height = df.loc[i, 'Sw_Rating'] * df.loc[i,'ADR'] / 75

    # set a limit on zone size 
    a = df.loc[2, 'DT'] - df.loc[1, 'DT']  
    b = pd.Timedelta(days = 1)
    cpd = b / a         # candles per day
    max_size = df.loc[i, 'ADR'] / cpd * 42  # wont work for D1 TF 

    if zone_height > max_size:
        zone_height = max_size
    # print("zone height: ", zone_height, "\n max: ", max_size)
    if df.loc[i, 'Sw_Size'] < 0:       # swing high
        df.loc[i,'TZ_Type'] = 'sell'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price'] + zone_height     # upper price 
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price']     # lower    
    else:               # swing low
        df.loc[i,'TZ_Type'] = 'buy'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price']     
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price'] - zone_height    
    
    ####  set end  ####
    end = round(df.loc[i, 'Sw_Rating'] * 3.5 + i)   # now get the length of the zone
    if end >  len(df) - 1:      # set max value as the length of df
        end =  len(df) - 1
    # elif end > round(max_size * 10000):
    #     end = round(max_size * 10000)
    df.loc[i,'TZ_End'] = end
   
    ####  set a custom start  ####
    start = i # + round((end - i) * 0.2) (makes viz difficult)
    if start >  len(df) - 1:
        start =  len(df) - 1
    df.loc[i,'TZ_Start'] = start

    ####  set buffers  ####  (used for switching zone type)
    df['Buf_Hi'] = df['TZ_H'] + (df['TZ_H'] - df['TZ_L']) * 0.25
    df['Buf_Lo'] = df['TZ_L'] - (df['TZ_H'] - df['TZ_L']) * 0.25

    ####  df for candles within current zone  ####
    zone_df = df.loc[start:end]
    zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                        (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
   

    if df.loc[i,'TZ_Type'] == 'sell':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles
                signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'up')]
                if len(signal_a) > 0:
                    # in case of multiple signals, return highest entry price
                    sell_price = max(signal_a['O'])
                    idx = signal_a['O'].idxmax() # get index of entry candle
                    date = zone_df.loc[idx, 'DT']

                    # plot 
                    trade_entries.append(sell_price)
                    dates.append(date)
                    names.append('sell')
                    plot_sellx.append(df.loc[start,'DT'])
                    plot_sellx.append(df.loc[end,'DT'])
                    plot_sellx.append(df.loc[end,'DT'])
                    plot_sellx.append(df.loc[start,'DT'])
                    plot_sellx.append(df.loc[start,'DT'])
                    plot_sellx.append(None)
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(df.loc[i,'TZ_H'])
                    plot_selly.append(df.loc[i,'TZ_H'])
                    plot_selly.append(df.loc[i,'TZ_L'])
                    plot_selly.append(None)

                # now check if zone gets breached to the upside
                new_high = zone_df['H'][zone_df['H'] > df.loc[i, 'Buf_Hi']]
                if len(new_high) > 0: 
                    df.loc[i,'TZ_Type'] = 'buy' # change zone type
                    start = min(new_high.index)    # shift start forward to first of breach candles
                    end = round((end - start) * 0.5 + end)     # shift end forward by 50%
                    if end > len(df) - 1:
                        end = len(df) - 1
                    df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 # shift zone down by 50%
                    df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

                    # reset the zone_df to reflect new zone dimensions
                    flipped_df = df.loc[start:end]
                    flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                    (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]

                    # and look for trades again, but this time buys. if zone
                    # gets breached for a second time it becomes inactive   

                    new_low = flipped_df['L'][flipped_df['L'] < df.loc[i, 'TZ_L']]   
                    if len(flipped_df) > 0:     # first see if any candles exist in new zone
                        if len(new_low) > 0:    # then see if it got breached 
                            idx = min(new_low.index)    # irst candle to breach
                            df.loc[i,'TZ_Active'] = False  
                            # shorten the flipped_df to end at that candle
                            # (this won't be relevent for live data, but it's necessary
                            #  to backtest like this. for live data, once the zone
                            #  becomes inactive it will simply be ignored and any
                            #  trades by that point would have already been found)
                            flipped_df = flipped_df.loc[start:idx]
                        signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'down')]
                        if len(signal_a) > 0:
                            # in case of multiple signals, get lowest price for buy 
                            buy_price = min(signal_a['O'])
                            idx = signal_a['O'].idxmin()  
                            date = df.loc[idx, 'DT']
                            
                            # plot 
                            trade_entries.append(buy_price)
                            dates.append(date)
                            names.append('buy')            
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(df.loc[end, 'DT'])
                            plot_sell_flipx.append(df.loc[end, 'DT'])
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(None)
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(None)
                
    if df.loc[i,'TZ_Type'] == 'buy':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles
                signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'down')]
                if len(signal_a) > 0:
                    # in case of multiple signals, return highest entry price
                    buy_price = min(signal_a['O'])
                    idx = signal_a['O'].idxmin() # get index of entry candle
                    date = zone_df.loc[idx, 'DT']

                    # plot 
                    trade_entries.append(buy_price)
                    dates.append(date)
                    names.append('buy')
                    plot_buyx.append(df.loc[start,'DT'])
                    plot_buyx.append(df.loc[end,'DT'])
                    plot_buyx.append(df.loc[end,'DT'])
                    plot_buyx.append(df.loc[start,'DT'])
                    plot_buyx.append(df.loc[start,'DT'])
                    plot_buyx.append(None)
                    plot_buyy.append(df.loc[i,'TZ_L'])
                    plot_buyy.append(df.loc[i,'TZ_L'])
                    plot_buyy.append(df.loc[i,'TZ_H'])
                    plot_buyy.append(df.loc[i,'TZ_H'])
                    plot_buyy.append(df.loc[i,'TZ_L'])
                    plot_buyy.append(None)

                # now check if zone gets breached to the downside
                new_low = zone_df['L'][zone_df['L'] < df.loc[i, 'Buf_Lo']]
                if len(new_low) > 0: 
                    df.loc[i,'TZ_Type'] = 'sell' # change zone type
                    start = min(new_low.index)    # shift start forward to first of breach candles
                    end = round((end - start) * 0.5 + end)     # shift end forward by 50%
                    if end > len(df) - 1:
                        end = len(df) - 1
                    df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 # shift zone down by 50%
                    df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

                    # reset the zone_df to reflect new zone dimensions
                    flipped_df = df.loc[start:end]
                    flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                    (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]

                    # and look for trades again, but this time buys. if zone
                    # gets breached for a second time it becomes inactive   

                    new_high = flipped_df['H'][flipped_df['H'] > df.loc[i, 'TZ_H']]   
                    if len(flipped_df) > 0:     # first see if any candles exist in new zone
                        if len(new_high) > 0:    # then see if it got breached 
                            idx = min(new_high.index)    # irst candle to breach
                            df.loc[i,'TZ_Active'] = False  
                            # shorten the flipped_df to end at that candle
                            # (this won't be relevent for live data, but it's necessary
                            #  to backtest like this. for live data, once the zone
                            #  becomes inactive it will simply be ignored and any
                            #  trades by that point would have already been found)
                            flipped_df = flipped_df.loc[start:idx]
                        signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'up')]
                        if len(signal_a) > 0:
                            # in case of multiple signals, get lowest price for buy 
                            sell_price = max(signal_a['O'])
                            idx = signal_a['O'].idxmax()  
                            date = df.loc[idx, 'DT']
                            
                            # plot 
                            trade_entries.append(buy_price)
                            dates.append(date)
                            names.append('sell')            
                            plot_buy_flipx.append(df.loc[start, 'DT'])
                            plot_buy_flipx.append(df.loc[end, 'DT'])
                            plot_buy_flipx.append(df.loc[end, 'DT'])
                            plot_buy_flipx.append(df.loc[start, 'DT'])
                            plot_buy_flipx.append(df.loc[start, 'DT'])
                            plot_buy_flipx.append(None)
                            plot_buy_flipy.append(df.loc[i,'TZ_L'])
                            plot_buy_flipy.append(df.loc[i,'TZ_L'])
                            plot_buy_flipy.append(df.loc[i,'TZ_H'])
                            plot_buy_flipy.append(df.loc[i,'TZ_H'])
                            plot_buy_flipy.append(df.loc[i,'TZ_L'])
                            plot_buy_flipy.append(None)



#### final invalidation of trade zones, by expiry
# if current bar in df is past TZ_End, zone is expired
# df['TZ_Active'][df.index.loc[-1] > df['TZ_End'] = False    # uncomment for live data

# plot_buyx = [] 
# plot_buyy = []
# plot_buy_flipx = [] 
# plot_buy_flipy = []
# plot_sellx = [] 
# plot_selly = []
# plot_buy_flipx = [] 
# plot_sell_flipy = []
# trade_entries = []
# dates = []
# names = []

fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])

# current zones in purple
fig.add_trace(go.Scatter(x=plot_buyx, y=plot_buyy, fill="toself", fillcolor='green', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_buy_flipy, fill="toself", fillcolor='rgba(0, 100, 0, 1)', opacity=.4))
fig.add_trace(go.Scatter(x=plot_sellx, y=plot_selly, fill="toself", fillcolor='red', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_sell_flipy, fill="toself", fillcolor='rgba(100, 0, 0, 1)', opacity=.4))


fig.add_trace(go.Scatter(
    x=dates,
    y=trade_entries,
    mode="markers+text",
    name="Swing Ratings",
    text=names,
    textposition="top center",
    # fillcolor='blue'
    marker_size=10,
    marker_color='rgba(0, 0, 0, 1)'
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


# Trade Setup


In [0]:
active_zones = df[(df['TZ_H'] != np.nan) & # there is a zone
                  (df['TZ_Active'] != False)] 
trade_entries = []
dates = []
names = []
# for each zone, find candles that exist within its boundaries
for i in active_zones.index:

    start = df.loc[i,'TZ_Start']
    end = df.loc[i,'TZ_End']
    high = df.loc[i,'TZ_H']
    low = df.loc[i,'TZ_L']
    temp = df.loc[start:end]

    temp = temp[(temp['H'].between(low, high)) |
                (temp['L'].between(low, high))]
    
    
    # if candles are found, look for a candle with lower vol than 
    # original swing, or a series of 3 sequentially lower vol candles
    if len(temp) > 0:
        if df.loc[i,'TZ_Type'] == 'sell': 
            signal_a = temp[(temp['V'] < df.loc[i,'V']) & (temp['Bar'] == 'up')]
            signal_b = 1 # how to code 3 sequentially lower vols? 
            
            if len(signal_a) > 0:
                # in case of multiple signals, return highest
                sell_price = max(signal_a['O'])
                trade_entries.append(sell_price)
                idx = signal_a['O'].idxmax
                date = df.loc[idx, 'DT']
                dates.append(date)
                names.append('sell')
       
        if df.loc[i,'TZ_Type'] == 'buy': 
            signal_a = temp[(temp['V'] < df.loc[i,'V']) & (temp['Bar'] == 'down')]
            if len(signal_a) > 0:
                buy_price = min(signal_a['O'])
                trade_entries.append(buy_price)
                idx = signal_a['O'].idxmin
                date = df.loc[idx, 'DT']
                dates.append(date)
                names.append('buy')
                

# plot zones

fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])


fig.add_trace(go.Scatter(
    x=dates,
    y=trade_entries,
    mode="markers+text",
    name="Swing Ratings",
    text=names,
    textposition="top center",
    # fillcolor='blue'
    marker_size=10,
    marker_color='rgba(0, 0, 0, 1)'
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


# Confidence Ratings

In [0]:
####  current trend  ####
# last_5 = df[df['Sw_Rating'] > 3]


Visual plot: S/R and trade zones

In [0]:
####    S/R Zones    #####

# save the zones for charting 
plotx = [] # valid zones
ploty = []
plotxx = [] # invalidated zones
plotyy = []

for i in range(0, len(df)):
  swing = df.loc[i, 'Sw_Rating']

  # get the size of price zone around a swing
  # to change size adjust ADR denominator
  price_box = round(swing * df.loc[i,'ADR']/200, 4) 
  # upper price
  upper = df.loc[i,'Sw_Price'] + price_box / 2
  # lower price
  lower = df.loc[i,'Sw_Price'] - price_box / 2

  # get the time slice 
  end = swing * 10 + i # n bars forward from current bar

  # needs to be done:
  # 
  # if there is a prior swing next to the main one, and if the 
  # zone can connect to it without being intersected by price,
  # set the zone start to be the preceeding swing while maintaining
  # the Sw_Rating of the larger swing
  

  # find all swing prices that fall within those boundaries
  temp = df.loc[i:end,['Sw_Price','Sw_Size','DT']]
  temp = temp[(temp['Sw_Price'] > lower) & (temp['Sw_Price'] < upper)]

  # if there's a zone
  if len(temp) > 1:
    

    # if swing high
    if temp.reset_index().iloc[0,2] < 0:
      hi_1 = temp.reset_index().loc[0, 'Sw_Price']
      hi_2 = temp.reset_index().loc[1, 'Sw_Price']
      expired = False
      # used for setting zone extensions
      diff = pd.to_datetime(max(temp['DT'])) - pd.to_datetime(min(temp['DT']))

      ### these next lines will determine if a zone has been invalidated ###
      # look for a new high above zone
      new_high = df.loc[i:][df.loc[i:, 'H'] > upper]

      # if found 
      if len(new_high) > 1:
        # look forwards for a new low below zone
        idx = new_high.index[0]+1
        new_low = df.loc[idx:][df.loc[idx:, 'L'] < lower]
        # if found
        if len(new_low) > 1:
          # zone is invalidated  -- will plot a certain color and terminate
          # on the chart at the point of the candle that broke it
          plotxx.append(min(temp['DT']))
          plotxx.append(min(new_low['DT'])) # first low that breaches 
          plotxx.append(min(new_low['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(None)
          # full zone
          plotyy.append(lower)
          plotyy.append(lower)
          plotyy.append(upper)
          plotyy.append(upper)
          plotyy.append(lower)
          plotyy.append(None)
          expired = True
      # otherwise plot a regular extended zone
      if not expired:
        plotx.append(min(temp['DT'])) 
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(min(temp['DT']))        
        plotx.append(min(temp['DT']))
        plotx.append(None)
        ploty.append(lower)
        ploty.append(lower)
        ploty.append(upper)        
        ploty.append(upper)
        ploty.append(lower)
        ploty.append(None)



    # if swing low
    if temp.reset_index().iloc[0,2] > 0:
      low_1 = temp.reset_index().loc[0, 'Sw_Price']
      low_2 = temp.reset_index().loc[1, 'Sw_Price']
      expired = False
      # used for setting zone extensions (not currently working)
      diff = pd.to_datetime(max(temp['DT'])) - pd.to_datetime(min(temp['DT']))

      # look for a new low below zone
      new_low = df.loc[i:][df.loc[i:, 'L'] < lower]

      # if found 
      if len(new_low) > 1:
        # look forwards for a new high above zone
        idx = new_low.index[0]+1
        new_high = df.loc[idx:][df.loc[idx:, 'H'] > upper]
        # if found
        if len(new_high) > 1:

          # zone is invalidated  -- will plot a certain color and terminate
          # on the chart at the point of the candle that broke it
          plotxx.append(min(temp['DT']))
          plotxx.append(min(new_high['DT'])) # first low that breaches 
          plotxx.append(min(new_high['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(None)
          # full zone
          plotyy.append(lower)
          plotyy.append(lower)
          plotyy.append(upper)
          plotyy.append(upper)
          plotyy.append(lower)
          plotyy.append(None)
          expired = True

      # otherwise plot a regular extended zone
      if not expired:
        plotx.append(min(temp['DT']))
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(min(temp['DT']))        
        plotx.append(min(temp['DT']))
        plotx.append(None)
        ploty.append(lower)
        ploty.append(lower)
        ploty.append(upper)        
        ploty.append(upper)
        ploty.append(lower)
        ploty.append(None)

    # # return sum and count of all swing ratings within window
    # df.loc[i,'SR_Sum'] = sum(df.loc[start:end,'Sw_Rating'])
    # df.loc[i, 'SR_Count'] = len(temp)
    # # print(temp)
